In [3]:
import pandas as pd

file_path = "US_Accidents_March23.csv"  # Adjust filename if needed
output_file = "US_Accidents_100K.csv"

# Define chunk size and sample fraction
chunk_size = 50000  # Read in chunks
sample_fraction = 100000 / 7700000  # 100K out of 7.7M rows

# Initialize an empty list to store sampled data
sampled_chunks = []

# Read in chunks and sample
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    sampled_chunk = chunk.sample(frac=sample_fraction, random_state=42)  # Randomly sample from each chunk
    sampled_chunks.append(sampled_chunk)  # Store the sample

# Combine all sampled chunks
df_sample = pd.concat(sampled_chunks)

# Save to CSV
df_sample.to_csv(output_file, index=False)

print(f"Saved {len(df_sample)} random rows to {output_file}")


Saved 100315 random rows to US_Accidents_100K.csv
